In [2]:
# from transformers import BloomTokenizerFast, BloomForQuestionAnswering, BloomForCausalLM, TrainingArguments, Trainer
import os
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from config.config import BASE_DIR, DATA_DIR, EMBEDDING_MODEL_NAME, PINECONE_INDEX_NAME


/Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()

True

In [4]:

# Save it into pinecone
API_KEY = os.environ.get("PINECONE_API_KEY")
YOUR_ENV = os.environ.get("PINECONE_ENVIRONMENT", "us-west4-gcp-free")
index_name = PINECONE_INDEX_NAME
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")

In [5]:
pinecone.init(
    api_key=API_KEY,
    environment=YOUR_ENV
)

if len(pinecone.list_indexes()) == 0:
    pinecone.create_index(name=index_name, metric="cosine", shards=1, dimension=len(res[0]))

pinecone.describe_index(pinecone.list_indexes()[0])

IndexDescription(name='stab-test', metric='cosine', replicas=1, dimension=768.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [6]:
text_field = "text"

index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 2678}},
 'total_vector_count': 2678}

In [7]:
embeddings = HuggingFaceEmbeddings(model_name = EMBEDDING_MODEL_NAME)

In [8]:
vectorstore = Pinecone(index, embeddings.embed_query, text_field)

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [25]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(kwargs={"pages": "pages"})
)

In [10]:
query_NVIDIA = "What is NVIDIa?"


In [18]:
vectorstore.max_marginal_relevance_search(
    "Importance of Agriculture and Cost-Effective Farm Policy",  # our search query
    k=5  # return 3 most relevant docs
)

[Document(page_content="assistance when factors beyond the producer's control create losses for producers. We \ngenerally think mo re tailored farm policies are more defendable.   For this reason, we like the \nthought of updating bases and yields or applying farm policies to planted acres/current \nproduction and their triggering based on prices or revenue, depending on the option a \nprod ucer chooses.  However, policy choices should not result in severe regional distortions in \ncommodity policy budget baselines from which reauthorized commodity policies must be \ndeveloped.", metadata={'chunk': 1.0, 'id': '721411c7627f4e63a303d595019038da', 'page': '7', 'source': 'ALQMFHOFCDKV7HIO2VGHCM3ZDDVVZPBW.pdf'}),
 Document(page_content='Member - USA Rice Pr oducers’ Group Farm Policy Task Force   \n   Member – USA Rice Federation Marketability and Competitiveness T ask Force \n    EDUCATION  \n 1999   Southeast Missouri State University – Cape Girardeau, MO  \nB.S. Agriculture with emphasis

In [14]:
response = qa_with_sources(query_NVIDIA)

In [15]:
response['answer'], response['sources']

('NVIDIA is not mentioned in the provided content.\nSOURCES:', '')

In [26]:
query_disambiguation = "How could i implement a disambiguation mechanism?"

In [27]:
response = qa_with_sources(query_disambiguation)

In [18]:
response['answer'], response['sources']

('To implement a disambiguation mechanism, one could consider using a cache model of attentional state. The cache model takes into account limited attention constraints and makes specific processing predictions. It has been proposed as an alternative to the stack model of attentional state, which does not account for certain examples. Future research could further distinguish between the predictions of the cache model and the stack model. Additionally, evidence for limited attention can be found in anaphoric processing and informational redundancy. These factors suggest that the limited attention constraint must be sensitive to aspects of linear recency. Further research and investigation are needed to fully understand and implement a disambiguation mechanism.\n',
 '9512003.xml, 9503008.xml')

In [27]:
query_senior_living = "I have some questions about senior living, what can you tell me about it? "

In [28]:
response = qa_with_sources(query_senior_living)

In [29]:
response['answer'], response['sources']

('Senior living is a social model that provides care while preserving independence. It is not the same as a nursing home, which primarily provides nursing services to the chronically ill. Senior living offers various lifestyle options for older adults who want to maintain their independence while living in their own apartments. Moving into senior living does not mean losing independence; in fact, it can enhance independence with assistance. The affordability of senior living can vary depending on factors such as location, level of care required, amenities offered, and the specific community chosen. Senior living offers benefits such as socializing on your terms, a supportive environment with trained staff, safety, security, and technology features, chore-free living, and access to entertainment. When considering senior living, it is important to assess factors such as the need for good company, opportunities to socialize, safety concerns, and challenges with daily tasks and personal ca

In [30]:
query_espanol = "Dime acerca de la vida de la tercera edad"

In [31]:
response = qa_with_sources(query_espanol)

In [32]:
response['answer'], response['sources']

('No information is provided about the life of the elderly.\n', 'common.txt')

In [15]:
query_espanol = "What is PowerPoint?"
response = qa_with_sources(query_espanol)
response['answer'], response['sources']

('PowerPoint is a software program used for creating and presenting slide show presentations. It allows users to create slides with text, images, and multimedia elements, and provides tools for designing and formatting the slides. PowerPoint presentations can be displayed on a computer screen or projected onto a larger screen for an audience to view. \n\n',
 '9507002.xml, samplepptx.pptx')

In [22]:
query_espanol = "Who is the goal keeper of Real Madrid?"
response = qa_with_sources(query_espanol)
response['answer'], response['sources']

('The goal keeper of Real Madrid is Thibaut Courtois.\n', 'goalkeepers.csv')

In [23]:
response

{'question': 'Who is the goal keeper of Real Madrid?',
 'answer': 'The goal keeper of Real Madrid is Thibaut Courtois.\n',
 'sources': 'goalkeepers.csv'}

In [20]:
query_espanol = "What is the Importance of Agriculture and Cost-Effective Farm Policy?"
response = qa_with_sources(query_espanol)
response['answer'], response['sources']

('The importance of agriculture and cost-effective farm policy is to provide assistance to producers in loss situations and to tailor policies to the size of the loss. It is also important to maintain and improve crop insurance for all crops, including rice. Additionally, there should be options for different production regions to effectively meet the risk management needs of producers. The 2011 Farm Bill package provides a good framework for the 2012 Farm Bill. The details of the policy should not result in severe regional distortions in commodity policy budget baselines. The policy should eliminate pay limits and eligibility tests and should be plain, bankable, tailored, and defendable.\n',
 'ALQMFHOFCDKV7HIO2VGHCM3ZDDVVZPBW.pdf')